# Extração de dados do Github

Pesquisando por iniciativas/projetos que utilizam Dados Abertos Governamentais através da [API do Github](https://developer.github.com/v3/)

In [1]:
import requests
import pandas as pd
import time
import logging
import sys

In [14]:
search_strings = [
            'dados abertos',
            'dados abertos brasil',
            'dados abertos governo',
            'dados abertos governamentais',
            'dados governamentais',
            'dados publicos abertos',
            'dados do governo',
            'analise de dados do governo',
            'analise de dados governamentais',
            'portal de dados do governo',
            'portal de dados governamentais',
            'portal publico do governo',
            'portal de dados abertos do governo',
        ]

Para a acesso a alguns recursos da API do github é preciso se autenticar, como aumentar o limite de requisições. Informações sobre autenticação podem ser encontradas [aqui](https://developer.github.com/v3/#authentication).

In [5]:
credentials = ('<user_name>','<token>')

Limite de requisições sem autenticação

In [6]:
t = requests.get('https://api.github.com/rate_limit')
t.json()

{'resources': {'core': {'limit': 60, 'remaining': 49, 'reset': 1580237255},
  'search': {'limit': 10, 'remaining': 10, 'reset': 1580236437},
  'graphql': {'limit': 0, 'remaining': 0, 'reset': 1580239977},
  'integration_manifest': {'limit': 5000,
   'remaining': 5000,
   'reset': 1580239977},
  'source_import': {'limit': 5, 'remaining': 5, 'reset': 1580236437}},
 'rate': {'limit': 60, 'remaining': 49, 'reset': 1580237255}}

Limite de requisições com autenticação

In [7]:
t = requests.get('https://api.github.com/rate_limit', auth=credentials)
t.json()

{'resources': {'core': {'limit': 5000, 'remaining': 5000, 'reset': 1580239978},
  'search': {'limit': 30, 'remaining': 26, 'reset': 1580236378},
  'graphql': {'limit': 5000, 'remaining': 5000, 'reset': 1580239978},
  'integration_manifest': {'limit': 5000,
   'remaining': 5000,
   'reset': 1580239978},
  'source_import': {'limit': 100, 'remaining': 100, 'reset': 1580236438}},
 'rate': {'limit': 5000, 'remaining': 5000, 'reset': 1580239978}}

Verificando limitação de extração de dados da API

In [8]:
page_35 = 'https://api.github.com/search/repositories?q=stars%3A%3E1&sort=stars&order=desc&page=35'
t = requests.get(page_35, auth=credentials)
t.json()

{'message': 'Only the first 1000 search results are available',
 'documentation_url': 'https://developer.github.com/v3/search/'}

Informações sobre a ferramenta de pesquisa da API podem ser encontradas [aqui](https://developer.github.com/v3/search/)

In [9]:
url_base = 'https://api.github.com/search/repositories?q='

Podemos adicionar uma ordenação nos resultados, como quantidade de _stars_ de forma descrescente.

In [10]:
sort = '&sort=stars&order=desc'

## Extraindo informações gerais

In [11]:
def extract_results(data):
    
    items_list = []
    
    for item in data.get('items', None):
        
        
        item_dict = {
                'id': item.get('id'),
                'full_name': item.get('full_name', None),
                'description': item.get('description', None),      
                'owner_type': item.get('owner').get('type', None),
                'owner_api_url': item.get('owner').get('url', None),
                'owner_url': item.get('owner').get('html_url', None),
                'api_url': item.get('url', None),
                'url': item.get('html_url', None),
                'fork': item.get('fork', None),
                'created_at': item.get('created_at', None),
                'updated_at': item.get('updated_at', None),
                'pushed_at': item.get('pushed_at', None),
                'size': item.get('size', None),
                'stargazers_count': item.get('stargazers_count', None),
                'language': item.get('language', None),
                'has_issues': item.get('has_issues', None),
                'has_wiki': item.get('has_wiki', None),
                'forks_count': item.get('forks_count', None),
                'forks': item.get('forks', None),
                'open_issues': item.get('open_issues', None),
                # 'subscribers_count': item.get('subscribers_count', None),
                'license': item.get('license').get('name', None) if item.get('license', None) else None,
                'timestamp_extract': str(time.time()).split('.')[0]
        }

        items_list.append(item_dict)
            
    return items_list

In [12]:
def scroll_pages(url):
        
    results = requests.get(url, auth=credentials)    
    data = results.json()
    total = data.get('total_count', None)
        
    print(">>> Foram encontrados {0} resultados. Extraindo...".format(total))
    
    items_list = []
    items_list = extract_results(data)
        
    iterations = total // 30 
    
    for iteracao in range(0, iterations):        
        header = dict(results.links)
        
        if header.get('next', False):
            next_url = header.get('next').get('url')
                        
            results = requests.get(next_url, auth=credentials)
            data = results.json()
            
            items_list.append(extract_results(data))
    
    return items_list

In [15]:
%%time

items_list = []
repositories_df = None

for string in search_strings:
    url = url_base + string + sort
    print("\nPesquisando repositórios para a string: '{0}'".format(string))
    
    items_list = items_list + scroll_pages(url)
        
repositories_df = pd.DataFrame(items_list)


Pesquisando repositórios para a string: 'portal de dados do governo'
>>> Foram encontrados 10 resultados. Extraindo...

Pesquisando repositórios para a string: 'portal de dados governamentais'
>>> Foram encontrados 2 resultados. Extraindo...

Pesquisando repositórios para a string: 'portal publico do governo'
>>> Foram encontrados 1 resultados. Extraindo...

Pesquisando repositórios para a string: 'portal de dados abertos do governo'
>>> Foram encontrados 5 resultados. Extraindo...
CPU times: user 88.3 ms, sys: 14 ms, total: 102 ms
Wall time: 2.6 s


In [16]:
repositories_df.describe()

,forks,forks_count,id,open_issues,size,stargazers_count
count,18.000000,18.000000,1.800000e+01,18.000000,18.000000,18.000000
mean,0.833333,0.833333,1.222096e+08,0.555556,3294.166667,2.166667
std,1.581139,1.581139,6.432144e+07,0.855585,4116.636352,4.218307
min,0.000000,0.000000,2.413158e+07,0.000000,1.000000,0.000000
25%,0.000000,0.000000,6.590246e+07,0.000000,146.000000,0.000000
50%,0.000000,0.000000,1.323673e+08,0.000000,3004.000000,0.000000
75%,1.000000,1.000000,1.859371e+08,1.000000,4397.750000,2.000000
max,5.000000,5.000000,2.157997e+08,2.000000,16753.000000,13.000000


In [17]:
repositories_df

,api_url,created_at,description,fork,forks,forks_count,full_name,has_issues,has_wiki,id,...,open_issues,owner_api_url,owner_type,owner_url,pushed_at,size,stargazers_count,timestamp_extract,updated_at,url
0,https://api.github.com/repos/dadosgovbr/ckanex...,2016-11-22T11:22:49Z,Plugin / Tema do Portal Brasileiro de Dados Ab...,False,5,5,dadosgovbr/ckanext-dadosgovbr,True,True,74465492,...,1,https://api.github.com/users/dadosgovbr,Organization,https://github.com/dadosgovbr,2019-06-05T18:18:50Z,5061,13,1580236446,2019-06-05T18:18:52Z,https://github.com/dadosgovbr/ckanext-dadosgovbr
1,https://api.github.com/repos/thenets/ckanext-d...,2016-08-17T11:33:20Z,Plugin / Tema do Portal de Dados Abertos do Go...,False,1,1,thenets/ckanext-dadosabertos,True,False,65902460,...,0,https://api.github.com/users/thenets,User,https://github.com/thenets,2016-11-22T11:47:38Z,3998,2,1580236446,2017-08-28T09:53:00Z,https://github.com/thenets/ckanext-dadosabertos
2,https://api.github.com/repos/mtrpires/raspafam...,2014-09-17T05:45:14Z,Raspador de dados do Portal da Transparência d...,False,0,0,mtrpires/raspafamilia,True,True,24131578,...,0,https://api.github.com/users/mtrpires,User,https://github.com/mtrpires,2014-09-17T14:04:44Z,136,2,1580236446,2018-03-07T00:04:05Z,https://github.com/mtrpires/raspafamilia
3,https://api.github.com/repos/edsonlead/gastos-...,2018-05-28T15:56:42Z,"Raspagem de dados, do portal da transparência,...",False,1,1,edsonlead/gastos-gov-federal,True,True,135181861,...,0,https://api.github.com/users/edsonlead,User,https://github.com/edsonlead,2018-06-02T01:41:15Z,2049,1,1580236446,2018-06-02T01:41:16Z,https://github.com/edsonlead/gastos-gov-federal
4,https://api.github.com/repos/Macmod/PortalTran...,2019-07-17T00:29:43Z,Extrator de dados do portal da transparência d...,False,0,0,Macmod/PortalTransparenciaBR,True,True,197286043,...,0,https://api.github.com/users/Macmod,User,https://github.com/Macmod,2019-07-17T14:12:54Z,21,0,1580236446,2019-07-17T14:12:56Z,https://github.com/Macmod/PortalTransparenciaBR
5,https://api.github.com/repos/prodest/dados.es....,2016-07-29T17:06:22Z,Portal de dados aberto do governo do ES,False,0,0,prodest/dados.es.gov.br,True,True,64497160,...,2,https://api.github.com/users/prodest,Organization,https://github.com/prodest,2016-07-29T17:06:26Z,14,0,1580236446,2016-07-29T17:06:22Z,https://github.com/prodest/dados.es.gov.br
6,https://api.github.com/repos/mateushtoledo/sysvis,2018-11-16T12:29:01Z,Repositório referente a uma aplicação de visua...,False,0,0,mateushtoledo/sysvis,True,True,157864726,...,0,https://api.github.com/users/mateushtoledo,User,https://github.com/mateushtoledo,2018-11-16T13:44:18Z,3004,0,1580236446,2018-11-16T13:44:22Z,https://github.com/mateushtoledo/sysvis
7,https://api.github.com/repos/prodest/ckanext-d...,2019-07-04T19:52:41Z,Plugin/Tema do Portal de Dados Abertos do Gove...,False,1,1,prodest/ckanext-data_es_theme,True,True,195294598,...,0,https://api.github.com/users/prodest,Organization,https://github.com/prodest,2019-07-24T17:52:46Z,176,0,1580236446,2019-07-24T17:52:48Z,https://github.com/prodest/ckanext-data_es_theme
8,https://api.github.com/repos/nosdeedson/Projet...,2019-10-17T13:30:36Z,O projeto faz a leitura de APIs do portal da t...,False,0,0,nosdeedson/Projeto-leitura-de-API-com-Docker,True,True,215799702,...,0,https://api.github.com/users/nosdeedson,User,https://github.com/nosdeedson,2019-10-21T12:52:26Z,4531,0,1580236446,2019-10-21T12:52:29Z,https://github.com/nosdeedson/Projeto-leitura-...
9,https://api.github.com/repos/Glautor/participe-db,2018-07-17T04:55:56Z,Banco de dados em servidor REST criado para ar...,False,0,0,Glautor/participe-db,True,True,141233148,...,0,https://api.github.com/users/Glautor,User,https://github.com/Glautor,2018-07-31T02:48:20Z,8187,0,1580236446,2018-07-31T02:48:22Z,https://github.com/Glautor/participe-db


## Extraindo _Commits_, _Contributors_ e dados do _Owner_

In [ ]:
def extract_commits(url_repo):
    
    commits_url = url_repo + '/commits'  
    results = requests.get(commits_url, auth=credentials)
    
    # No caso do repositório estar vazio
    if results.status_code == 409:
        return None
    
    commits = len(results.json())

    header = dict(results.links)
    
    while header.get('next', False):
        next_url = header.get('next').get('url')        
        results = requests.get(next_url, auth=credentials)
        commits = commits + len(results.json())    
        header = dict(results.links)


    return commits

In [ ]:
def extract_contributors(url_repo):
    
    contributors_url = url_repo + '/contributors'
    results = requests.get(contributors_url, auth=credentials)
    
    # No caso do repositório estar vazio
    if results.status_code == 204:
        return None
    
    contributors = len(results.json())

    header = dict(results.links)
    
    while header.get('next', False):
        next_url = header.get('next').get('url')
        results = requests.get(next_url, auth=credentials)
        contributors = contributors + len(results.json())
        header = dict(results.links)
    
    return contributors

In [ ]:
def extract_owner_data(owner_api_url):
    
    results = requests.get(owner_api_url, auth=credentials)
    data = dict(results.json())

    owner_data = {
        'owner_location': data.get('location', None),
        'owner_email': data.get('email', None),
        'owner_blog': data.get('blog', None),
        'owner_name': data.get('name', None)
    }
    
    return owner_data

In [ ]:
%%time
urls = repositories_df['api_url']

for url in urls:
            
    repo = requests.get(url, auth=credentials)
    repo = dict(repo.json())
        
    commits = extract_commits(url)
    contributors = extract_contributors(url)

    repositories_df.loc[repositories_df["api_url"] == url, 'commits'] = commits
    repositories_df.loc[repositories_df["api_url"] == url, 'contributors'] = contributors

In [ ]:
repositories_df.head()

Conferindo valores nulos

In [ ]:
repositories_df.loc[repositories_df['commits'].isnull()][['api_url', 'commits', 'contributors']]

In [ ]:
repositories_df.loc[repositories_df['contributors'].isnull()][['api_url', 'commits', 'contributors']]

In [ ]:
file_path = '../data/repositories_' + str(time.time()).split('.')[0] + '.csv'
file_path

In [ ]:
repositories_df.to_csv(file_path, index=False)

## Extraindo contribuidores dos repositórios

In [ ]:
# todo: Extrair locations

In [ ]:
columns_contributors = [ 'repo_id',
                          'repo_name',
                          'repo_url',
                          'repo_api_url',
                          'contributor_id',
                          'contributor_login',
                          'contributor_type',
                          'contributor_url',
                          'contributor_api_url',
                          'timestamp_extract']

In [ ]:
def scroll_results(results):
    
    list_contributors = []

    for result in results:
        contributor = {}
        
        contributor = {
            'contributor_id': result.get('id', None),
            'contributor_login': result.get('login', None),
            'contributor_type': result.get('type', None),
            'contributor_url': result.get('html_url', None),
            'contributor_api_url': result.get('url', None),
        }

        list_contributors.append(contributor)
        
    return list_contributors

In [ ]:
def get_contributors(url):

    list_contributors = []
    results = requests.get(url, auth=credentials)
    
    # No caso do repositório estar vazio
    if results.status_code is 204:
        return None
    
    contributors_results = results.json()
    list_contributors = scroll_results(contributors_results)
    
    header = dict(results.links)
    
    while header.get('next', False):
        next_url = header.get('next').get('url')
        
        print('\t> Extraindo da próxima página: {0}'.format(next_url))
            
        results = requests.get(next_url, auth=credentials)
        contributors_results = results.json()
        list_contributors = list_contributors + scroll_results(contributors_results)    
        header = dict(results.links)
        
    return list_contributors

In [ ]:
def add_contributor(repo, contributor):
            
    df = pd.DataFrame([[
                    repo.get('repo_id', None),
                    repo.get('repo_name', None),
                    repo.get('repo_url', None),      
                    repo.get('repo_api_url', None),
                    contributor.get('contributor_id', None),
                    contributor.get('contributor_login', None),
                    contributor.get('contributor_type', None),
                    contributor.get('contributor_url', None),
                    contributor.get('contributor_api_url', None),
                    str(time.time()).split('.')[0]]], columns=columns_contributors)

    return df  

In [ ]:
def save_contributors(contributors_df, repo, contributors):
    
    for contributor in contributors:
                
        contributors_df = pd.concat([contributors_df, 
                                     add_contributor(repo, contributor)], 
                                    ignore_index=True, 
                                    sort=False)
    return contributors_df

In [ ]:
def search_contributors(repositories_df):
    
    contributors_df = pd.DataFrame(columns = columns_contributors)
    urls = repositories_df['api_url']
   
    for url in urls:
        print('\nExtraindo contribuidores de: {0}'.format(url))

        repo = {
            'repo_id': repositories_df.loc[repositories_df["api_url"] == url, 'id'].values[0],
            'repo_name': repositories_df.loc[repositories_df["api_url"] == url, 'full_name'].values[0],
            'repo_url': repositories_df.loc[repositories_df["api_url"] == url, 'url'].values[0],
            'repo_api_url': url,
        }
        
        url_contributors = url + '/contributors'        
        contributors = get_contributors(url_contributors)
        
        if contributors:
            contributors_df = save_contributors(contributors_df, repo, contributors)
        
    
    return contributors_df

In [ ]:
%%time
result_contributors = search_contributors(repositories_df)

In [ ]:
result_contributors.describe()

In [ ]:
result_contributors.head()

In [ ]:
file_path = '../data/contributors_' + str(time.time()).split('.')[0] + '.csv'
file_path

In [ ]:
result_contributors.to_csv(file_path, index=False)